# Analyzing borrowers’ risk of defaulting

The project is to prepare a report for a bank’s loan division. We'll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

The report will be considered when building the **credit scoring** of a potential customer. The credit scoring is used to evaluate the ability of a potential borrower to repay their loan.

## Step 1. Open the data file and have a look at the general information. 

In [1]:
import pandas as pd 

import sys
import warnings
if not sys.warnoptions:    
       warnings.simplefilter("ignore")

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemma = nltk.WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#location = '/some/local/path'
location = 'C:\\Users\\User\\Documents\\Coding School\\Practicum by Yandex\\3 Introduction to Data Preprocessing\\Project\\'
raw_data = pd.read_csv(location + 'credit_scoring_eng.csv')

In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
raw_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [5]:
raw_data.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car
21524,2,-1984.507589,40,secondary education,1,married,0,F,employee,0,13127.587,to buy a car


In [6]:
raw_data.sample()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17621,0,NaN,41,bachelor's degree,0,civil partnership,1,M,business,0,NaN,wedding ceremony


In [7]:
raw_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


### Stage summary:
Preliminary findings from examining the data using info, head, tail, sample, and describe methods:
1. unclear column names
2. children column- weird values (-1 and 20)
3. days_employed - corrupted data (negative values) and NaNs
4. dob_years - missing values (0)
5. education - includes duplicates due to mixed-case letters. Some are capital, and others are not.
6. income_type - NaN values + can be categorized (income/no income)
7. purpose - can be categorized 
8. Two columns with float64 data type that should be int64

## Step 2. Data preprocessing

### Processing missing values

##### Children

In [8]:
raw_data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [9]:
raw_data[raw_data['children'] == -1].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,secondary education,1,civil partnership,1,F,employee,0,16450.615,profile education
705,-1,-902.084528,50,secondary education,1,married,0,F,civil servant,0,22061.264,car purchase
742,-1,-3174.456205,57,secondary education,1,married,0,F,employee,0,10282.887,supplementary education
800,-1,349987.852217,54,secondary education,1,unmarried,4,F,retiree,0,13806.996,supplementary education
941,-1,NaN,57,Secondary Education,1,married,0,F,retiree,0,NaN,buying my own car
1363,-1,-1195.264956,55,SECONDARY EDUCATION,1,married,0,F,business,0,11128.112,profile education
1929,-1,-1461.303336,38,secondary education,1,unmarried,4,M,employee,0,17459.451,purchase of the house
2073,-1,-2539.761232,42,secondary education,1,divorced,3,F,business,0,26022.177,purchase of the house
3814,-1,-3045.290443,26,Secondary Education,1,civil partnership,1,F,civil servant,0,21102.846,having a wedding
4201,-1,-901.101738,41,secondary education,1,married,0,F,civil servant,0,36220.123,transactions with my real estate


In [10]:
raw_data[raw_data['children'] == 20].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,secondary education,1,married,0,M,business,0,23253.578,purchase of the house
720,20,-855.595512,44,secondary education,1,married,0,F,business,0,18079.798,buy real estate
1074,20,-3310.411598,56,secondary education,1,married,0,F,employee,1,36722.966,getting an education
2510,20,-2714.161249,59,bachelor's degree,0,widow / widower,2,F,employee,0,42315.974,transactions with commercial real estate
2941,20,-2161.591519,0,secondary education,1,married,0,F,employee,0,31958.391,to buy a car
3302,20,NaN,35,secondary education,1,unmarried,4,F,civil servant,0,NaN,profile education
3396,20,NaN,56,bachelor's degree,0,married,0,F,business,0,NaN,university education
3671,20,-913.161503,23,secondary education,1,unmarried,4,F,employee,0,16200.879,buying a second-hand car
3697,20,-2907.910616,40,secondary education,1,civil partnership,1,M,employee,0,18460.911,buying a second-hand car
3735,20,-805.044438,26,bachelor's degree,0,unmarried,4,M,employee,0,21952.103,housing renovation


Using the describe() method, I noticed that the min value in the column is -1 and the max value is 20. Both are weird values for the number of children. Using the value_counts() method, I can see the number of problematic rows with these values. I can also see that these are the only missing values in the column as 0 is a valid value for this column, and there are no NaNs.  
Checking the first 5 rows with each of those values, I couldn't find any common factors.  
-1 child is an impossible value.  
20 children are possible, though highly unlikely. Even if there are applicants with 20 children, it isn't plausible that there are 76 such entries.  
This leads me to conclude that both values are typos and should be changed to 1 and 2 respectively using the replace() method.  

In [11]:
raw_data['children'] = raw_data['children'].replace(-1 , 1)
raw_data['children'] = raw_data['children'].replace(20 , 2)
raw_data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [12]:
raw_data.head() ## test

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


##### days_employed column

Using the info() method in the previous step I noticed that there is the same amount of NaNs in the 'days_employed' and 'total_income' columns. I assume that every table row that has a NaN in the 'days employed' column also has a NaN in the 'total_income' column.  
It makes sense since a person that didn't work (or didn't report his employment record) doesn't have a valid income value.  
Let's verify this assumption by counting first the number of rows where there's a missing value in 'days_employed' then compare to number of rows where both values are missing. 

In [13]:
raw_data[raw_data['days_employed'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


In [14]:
num_of_missing_days_employed = raw_data.isnull().sum()['days_employed']
a = raw_data.loc[:,'days_employed'].isnull()
b = raw_data.loc[:,'total_income'].isnull()
c = a & b
print(c.sum() == num_of_missing_days_employed)
print('Amount of missing values in employment related columns:', num_of_missing_days_employed)

True
Amount of missing values in employment related columns: 2174


Since the missing rows amount to 10% of all data and the 'days_employed' column will not affect the final conclusions, I think the NaNs in this column should be filled with '0'. This correctly reflects my hypothesis that loan applicants purposely avoided this column.   
I used the fillna() method. 

In [15]:
raw_data['days_employed'] = raw_data['days_employed'].fillna(0)
raw_data.isnull().sum()['days_employed']

0

In [16]:
raw_data.head() ## test

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


An additional issue with the days_employed column is the negative values and large positive values.  
It is apparent that the negative values in this column correspond with the income_type values (employed applicants have negative days_employes values). I suppose this column stands for the number of days that the applicant is employed at their current job. The enormous values for unemployed and retired applicants stand for NaN.  

Cosidering that this column won't be used in further analysis, I will conver all negative values to positive and leave the large positive value as is.    

In [17]:
raw_data['days_employed'] = raw_data['days_employed'].abs()
raw_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


##### dob_years column:

In [18]:
raw_data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

In [19]:
age_median = int(raw_data['dob_years'].median())
print('The median age is:', age_median)
raw_data['dob_years'] = raw_data['dob_years'].replace(0, age_median)
raw_data['dob_years'].unique()

The median age is: 42


array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

There are 101 rows with 0 in this column. I assume that the applicant left this parameter out. This value should be filled with the median age.  
I saved the median age to the variable age_median after converting it to int type. Then I replaced all 0 values with that age_median variable.   

In [20]:
raw_data.head() ## test

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


##### education column: 

In [21]:
raw_data['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64

Using the value_counts() I can see that there are no missing values in this column. There are duplicates, but I will deal with those later.  

##### education_id column:

In [22]:
raw_data['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

0 is a legitimate value for this column. No missing values here. 

##### family_status column: 

In [23]:
raw_data['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

No missing values. 

##### family_status_id column: 

In [24]:
raw_data['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

same as education_id. No missing values. 

##### gender column: 

In [25]:
raw_data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

There is one row with a weird value: XNA. I think this value refers to non-specific gender. In Australia, for example, the guidelines give the option of selecting M (male), F (female), or X (Indeterminate/Intersex/Unspecified) for personal records.  
Since this column's values don't affect the final conclusion, I see no reason to change this.   

##### income_type column: 

In [26]:
raw_data['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

No missing values. 

##### debt column: 

In [27]:
raw_data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

No missing values. 0 is an acceptable value here. 

##### total_income column: 

The 'total_income' column in necessary and wrong values might corrupt the conclusions. For that reason, I decided to fill the NaNs with the median income for their income type. 

In [28]:
#create an array containing all income types that have nulls in their total income
null_income = raw_data[raw_data['total_income'].isnull()]
null_income_type = null_income['income_type'].unique()
null_income_type

array(['retiree', 'civil servant', 'business', 'employee', 'entrepreneur'],
      dtype=object)

In [29]:
# iterate over those income types and replace missing total income values with calculated median for each income type
for income_type in null_income_type:
    idx_by_income_type = (raw_data['income_type'] == income_type)
    idx_by_total_income_nulls = raw_data['total_income'].isnull() 
    income_type_df = raw_data[idx_by_income_type]
    median_for_income = income_type_df["total_income"].median() # nulls are skipped by default!
    print('The median income for {} is: {:.2f}'.format(income_type,median_for_income))
    raw_data.loc[idx_by_total_income_nulls & idx_by_income_type,"total_income"] = median_for_income

raw_data.isnull().sum()['total_income']   


The median income for retiree is: 18962.32
The median income for civil servant is: 24071.67
The median income for business is: 27577.27
The median income for employee is: 22815.10
The median income for entrepreneur is: 79866.10


0

In [30]:
raw_data.info() #test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [31]:
raw_data.head() ## yet another test

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


##### purpose column: 

In [32]:
raw_data['purpose'].value_counts()

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

No missing values. There are many repeating values (values that have the same meaning), and those will be handled in the duplicates section. 

#### Stage summary:

I noticed different kinds of bad data and handled it as described above. 

### Data type replacement

In [33]:
raw_data.head() ## reality check

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [34]:
raw_data['days_employed'] = raw_data['days_employed'].astype(int)
raw_data['total_income'] = raw_data['total_income'].astype(int)
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


#### Stage summary: 

I converted float type to int using the astype() method. 

### Processing duplicates

As we noticed above, the education column has a lot of duplicates due to use of both upper and lower cases inconsistently. To get rid of those duplicated values I converted all values to lower case using the str.lower() method. 

In [35]:
raw_data['education'] = raw_data['education'].str.lower()
raw_data['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [36]:
raw_data.head() # test

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding


Using the duplicated() and sum() methods I found that there are 72 row in the raw_data df that are duplicated. I consider them as duplicated because it is very unlikely that two diffrent applicants will have the exact same entries in every column. Duplications like this might happen because people apply for a loan several times. For this not to affect our conclusions, duplicated lines should be droped using the drop_duplicates() method. 

In [37]:
print('Number of duplicated rows:', raw_data.duplicated().sum())

Number of duplicated rows: 72


In [38]:
raw_data = raw_data.drop_duplicates().reset_index(drop=True)
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21453 non-null  int64 
 1   days_employed     21453 non-null  int32 
 2   dob_years         21453 non-null  int64 
 3   education         21453 non-null  object
 4   education_id      21453 non-null  int64 
 5   family_status     21453 non-null  object
 6   family_status_id  21453 non-null  int64 
 7   gender            21453 non-null  object
 8   income_type       21453 non-null  object
 9   debt              21453 non-null  int64 
 10  total_income      21453 non-null  int32 
 11  purpose           21453 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


In [39]:
raw_data.head(5) #just a test

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding


#### Stage summary:

As specified above. 

## Categorizing Data

In [40]:
raw_data.set_axis(['num_of_children','current_days_employed','age','education','education_id','marital_status','marital_status_id','gender','income_type','debt','annual_income','purpose'], axis = 'columns', inplace = True) 
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   num_of_children        21453 non-null  int64 
 1   current_days_employed  21453 non-null  int32 
 2   age                    21453 non-null  int64 
 3   education              21453 non-null  object
 4   education_id           21453 non-null  int64 
 5   marital_status         21453 non-null  object
 6   marital_status_id      21453 non-null  int64 
 7   gender                 21453 non-null  object
 8   income_type            21453 non-null  object
 9   debt                   21453 non-null  int64 
 10  annual_income          21453 non-null  int32 
 11  purpose                21453 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


For readability of the data I changes the column names using the set_axis() method. 

The next step is to categorize the annual income of the applicants. I categorized according to the U.S. Census Bureau break down of income levels. I wrote a function that returns the category and applied it to the df using the apply() method.

In [41]:
#defining a function to categorize anuual income and applying it to the df
def income_sorting(data): 
    if data <= 20000:
        group = 'poor'
    elif data > 20000 & data <= 44999:
        group = 'low income'
    elif data > 45000 and data <= 139999:
        group = 'middle class'
    elif data > 140000 and data <= 149999:
        group = 'upper middle class'
    elif data > 150000 and data <= 199000:
        group = 'high income'
    else:
        group = 'highst tax brackets'
    return group

raw_data['income_category'] = raw_data['annual_income'].apply(income_sorting)
raw_data['income_category'].value_counts()                       

low income    13697
poor           7756
Name: income_category, dtype: int64

In [42]:
raw_data.head() #testing 123

,num_of_children,current_days_employed,age,education,education_id,marital_status,marital_status_id,gender,income_type,debt,annual_income,purpose,income_category
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,low income
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,poor
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,low income
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,low income
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,low income


Using the same methods as above, I categorized marital status to couple/single. 

In [43]:
#defining a function to categorize marital status and applying it to the df
def marital_sorting(data): 
    if data == 'married' or data == 'civil partnership':
        group = 'couple'
    else:
        group = 'single'
    return group

raw_data['marital_category'] = raw_data['marital_status'].apply(marital_sorting)
raw_data['marital_category'].value_counts()                       

couple    16489
single     4964
Name: marital_category, dtype: int64

As I noticed above, there are a lot of duplicated meanings in this column. This column should be categorized. By reviewing the data, I can extract 4 distinct values: real estate, wedding, car, and education. Each value has keywords that repeat themselves.  
I want to create a new column with one of those values for each row. I will explain each step in the comments.   

In [44]:
#defining a function that will lemmatize the purpose column
def lemmatize_text(text):
    words = nltk.word_tokenize(text)
    return [lemma.lemmatize(w, pos = 'n') for w in words]

#calling the lemmatize_text function using the apply() method. The results are returnd into a new column 'lemmas'  
raw_data['lemmas'] = raw_data['purpose'].apply(lemmatize_text)
raw_data.head()

,num_of_children,current_days_employed,age,education,education_id,marital_status,marital_status_id,gender,income_type,debt,annual_income,purpose,income_category,marital_category,lemmas
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,low income,couple,"[purchase, of, the, house]"
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,poor,couple,"[car, purchase]"
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,low income,couple,"[purchase, of, the, house]"
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,low income,couple,"[supplementary, education]"
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,low income,couple,"[to, have, a, wedding]"


In [45]:
#defining a function that loops through a tuple of keywords, checks if keyword is in 'lemmas' and returns the purpose category
# according to a dictionary 
def translate(s):
    keyword_dict = {('education' , 'university','educated') : 'education' ,
                ('house' , 'property' , 'real', 'housing') : 'real estate' ,
                ('car',) : 'car',
                ('wedding',):'wedding'}
    for key in keyword_dict.keys():
        for word in key:
            if word in s:
                return keyword_dict[key]
    return 'other' 

raw_data['purpose_category'] = raw_data['lemmas'].apply(translate)
raw_data['purpose_category'].value_counts()

real estate    10811
car             4306
education       4013
wedding         2323
Name: purpose_category, dtype: int64

#### Stage summary:

I added 3 new category columns that will make it easier to analize the data and answer the questions presented by the bank. 

## Step 3. Answer these questions

#### Is there a relation between having kids and repaying a loan on time?

I used the groupby() method to group together the data according to number of children (since this is the parameter we are checking). Using the agg() method I calculate both the number of entries for each category (count()) and the number of debt in each category (sum()). Deviding the sum by count gave me the ratio of default. I formated the ration column to include only 2 decimals after the point and to show the precentage and then printed out the results sorted by the ratio column.

In [46]:
data_grouped_children = raw_data.groupby('num_of_children').debt.agg(['count','sum'])
data_grouped_children 

,count,sum
num_of_children,,
0,14090,1063
1,4855,445
2,2128,202
3,330,27
4,41,4
5,9,0


In [47]:
data_grouped_children['ratio'] = data_grouped_children['sum']  / data_grouped_children['count']
data_grouped_children['ratio'] = data_grouped_children['ratio'].map('{:.2%}'.format)
data_grouped_children.sort_values(by='ratio', ascending=False)

,count,sum,ratio
num_of_children,,,
4,41,4,9.76%
2,2128,202,9.49%
1,4855,445,9.17%
3,330,27,8.18%
0,14090,1063,7.54%
5,9,0,0.00%


#### Conclusion

The numbers shows that there is a higher chance to default on a loan if the applicant has children. There is a default rate of 7.5% for applicats without children. This rate grows once the applicant has children, but the amount of children matters only a little. The only exception to this is the families with 5 children. This category has a 0 default rate, which is an anomaly. This anomaly can be explained by a small sample size- there are only a few families with 5 children in the dataset so they can have 0 default % by pure chance.     

#### Is there a relation between marital status and repaying a loan on time?

I used the same methods as specified above to calculate and print out the ratio of default according to marital status.

In [48]:
data_grouped_marital = raw_data.groupby('marital_category').debt.agg(['count','sum'])
data_grouped_marital 

,count,sum
marital_category,,
couple,16489,1319
single,4964,422


In [49]:
data_grouped_marital['ratio'] = data_grouped_marital['sum']  / data_grouped_marital['count']
data_grouped_marital['ratio'] = data_grouped_marital['ratio'].map('{:.2%}'.format)
data_grouped_marital.sort_values(by='ratio', ascending=False)

,count,sum,ratio
marital_category,,,
single,4964,422,8.50%
couple,16489,1319,8.00%


#### Conclusion

It seems that couples have a better chance of repaying their loan, but not by much. 

#### Is there a relation between income level and repaying a loan on time?

In [50]:
data_grouped_income = raw_data.groupby('income_category').debt.agg(['count','sum'])
data_grouped_income

,count,sum
income_category,,
low income,13697,1098
poor,7756,643


In [51]:
data_grouped_income['ratio'] = data_grouped_income['sum']  / data_grouped_income['count']
data_grouped_income['ratio'] = data_grouped_income['ratio'].map('{:.2%}'.format)
data_grouped_income.sort_values(by='ratio', ascending=False)

,count,sum,ratio
income_category,,,
poor,7756,643,8.29%
low income,13697,1098,8.02%


### Conclusion

Looks like the data is homogeneous when it comes to income. All of the applicants have low incomes or are poor. It makes sense that lower income borrowers have a higher chance to default on a loan and we see it in the data (poor have 8.29% default rate, where as low income have a 8.01% rate). Since I have no higer income applicants to compare them to, I can't draw any conclusion from this data, except to say that low income applicants have an 8% default rate.   

#### How do different loan purposes affect on-time repayment of the loan?

To answer this question I used a pivot table that grouped the applicants by purpose and showed the number of applicats in each category (count) and the numver of applicants with debt (sum). Deviding the sum by count returned the ratio of default. I formated the ration column to include only 2 decimals after the point and to show the precentage and then printed out the results sorted by the ratio column.

In [52]:
purpose_data_pivot = raw_data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['count','sum'])
purpose_data_pivot

,count,sum
,debt,debt
purpose_category,,
car,4306,403
education,4013,370
real estate,10811,782
wedding,2323,186


In [53]:
purpose_data_pivot['ratio'] = purpose_data_pivot['sum'] / purpose_data_pivot['count']
purpose_data_pivot['ratio'] = purpose_data_pivot['ratio'].map('{:.2%}'.format)
purpose_data_pivot.sort_values(by='ratio', ascending=False)

,count,sum,ratio
,debt,debt,
purpose_category,,,
car,4306,403,9.36%
education,4013,370,9.22%
wedding,2323,186,8.01%
real estate,10811,782,7.23%


### Conclusion

The data shows that the safest loans are for the purpose of real estate (7.23% default). Weddings are the next safest loan (8.01%). Education and car loans have the highest chance of default (9.22% and 9.36% respectively). 

## Step 4. General conclusion

In the income brackets that were presented in this dataset, the best borrowers would be applicants without children who need a loan for real estate of wedding purposes. The diffrence between couples and single applicants is negligible.   